In [1]:
class Beam:
    def __init__(self, beam_id, c_idx, row, column, r_d, c_d) -> None:
        self.beam_id = beam_id
        self.c_idx = c_idx
        self.children = []
        self.row = row 
        self.column = column
        self.r_direction = r_d #change in row number
        self.c_direction = c_d
        self.at_end = False
        self.finished = False
        pass

    def update_beam(self,grid):
        num_rows = len(grid)
        num_col = len(grid[0])
        if self.at_end:
            for c in self.children:
                c.update_beam(grid)
        else:
            next_r = self.row + self.r_direction
            next_c = self.column + self.c_direction
            curr_r = self.row
            curr_c = self.column
            if 0 <= (next_r) and (next_r) < num_rows and 0 <= (next_c) and (next_c) < num_col:
                self.c_idx += 1
                self._update_direction(grid[next_r][next_c])
            else:
                self.at_end = True

    def _update_direction(self, grid_el):
        symbol = grid_el.symbol
        visiting = grid_el.visiting(self.c_idx, self.r_direction, self.c_direction)
        self.row += self.r_direction
        self.column += self.c_direction
        if visiting:
            if symbol ==  "." or (symbol == "-" and self.c_direction != 0) or (symbol == "|" and self.r_direction != 0):
                pass   
            elif symbol == "/":
                temp =  self.c_direction
                self.c_direction = -self.r_direction
                self.r_direction = -temp
            elif symbol == "\\":
                temp =  self.c_direction
                self.c_direction = self.r_direction
                self.r_direction = temp
            elif symbol == "-":
                self.children.append(Beam((self.beam_id + ".1"), self.c_idx, self.row, self.column, 0, -1))
                self.children.append(Beam((self.beam_id + ".2"), self.c_idx, self.row, self.column, 0, 1))
                self.c_direction == 0
                self.r_direction == 0
                self.at_end = True
            elif symbol == "|":
                self.children.append(Beam((self.beam_id + ".1"), self.c_idx, self.row, self.column, -1, 0))
                self.children.append(Beam((self.beam_id + ".2"), self.c_idx, self.row, self.column, 1, 0))
                self.c_direction == 0
                self.r_direction == 0
                self.at_end = True
        else:
            self.at_end = True
            self.c_direction == 0
            self.r_direction == 0

    def get_visiting(self):
        vis = []
        if not self.at_end:
            vis.append([self.row, self.column])
        for c in self.children:
            if not c.at_end:
                visited = c.get_visiting()
                for v in visited: 
                    vis.append(v)
        return vis

    def check_finished(self):
        if not self.at_end:
            return False
        for c in self.children:
            c.check_finished()
            if not c.finished:
                return False
        self.finished = True
        return True
    
class GridElement:
    def __init__(self, symbol, row, column):
        self.symbol = symbol
        self.row = row
        self.column = column
        self.incoming_r = []
        self.incoming_c = []
        self.energised = False
        self.incoming_count = []
    
    def visiting(self, count, r_direction, c_direction):
        if self.symbol != ".":
            if r_direction in self.incoming_r or c_direction in self.incoming_c:
                return False
            elif r_direction != 0:
                self.incoming_r.append(r_direction)
            elif c_direction != 0:
                self.incoming_c.append(c_direction)
            self.incoming_count.append(count)
        self.energised = True
        return True
            

In [2]:
grid = []
with open(r"input-16.txt", "r") as file:
#with open(r"test-input-16.txt", "r") as file:
    for r_num, row in enumerate(file):
        row = row.replace("\n", "")
        el = [c for c in row]
        g = []
        for c, e in enumerate(el):
            g.append(GridElement(e, r_num, c))
        grid.append(g)
    light_ended = False
    beam = Beam("1",0,0,-1,0,1)
    while(not light_ended):        
        beam.update_beam(grid)
        light_ended = beam.check_finished()

    vis = []
    counter = 0
    for g in grid:
        l = []
        for s in g:
            if s.energised:
                l.append("#")
                counter += 1
            else:
                l.append(s.symbol)
        vis.append(l)

    for v in vis:
        print(v)
    print("Part 1, grid points visited:", counter)

['#', '.', '.', '.', '#', '.', '#', '.', '.', '.', '.', '#', '.', '#', '\\', '#', '.', '.', '.', '.', '.', '.', '#', '.', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '.', '.', '.', '\\', '.', '#', '.', '/', '.', '\\', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '.', '#', '.', '.', '.', '.', '.', '.', '#', '.', '.', '.', '.', '.', '.', '.', '.', '/', '.', '.', '.', '.', '/', '.', '.', '.', '.', '.', '.', '.', '.', '-', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '.', '.']
['#', '.', '.', '|', '#', '.', '#', '.', '\\', '.', '.', '#', '.', '#', '.', '#', '.', '.', '.', '.', '.', '.', '#', '.', '#', '.', '.', '/', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '.', '.', '.', '.', '-', '.', '.', '.', '.', '.', '.', '.', '.',